# Prediction and Score

#### Imports

In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
import csv
import tensorflow as tf
from tensorflow.keras import utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tqdm import tqdm
np.random.seed(42)
tf.random.set_seed(42)
from keras.models import load_model
from keras.utils import np_utils

#### Making the Dataset.csv

In [10]:
csv_col_1_header="path"
csv_col_2_header="hair_color"
path_to_val_img="./val_img/"
with open('dataset.csv','w',newline='') as new_file:
    csv_writer=csv.writer(new_file)
    csv_writer.writerow([csv_col_1_header,csv_col_2_header])
    
    # 'folders' variable lists the folders
    # later the index of the folder name in 'folders' variable
    # is used to write the lable of the image
    # eg. if folders[1] has value "Black" then in the csv for Hair-Color
    # the hair_color value will be 1
    
    # if the indices of the folders do not match the 'digital2label' variable
    # in the training file then manually pass the list in 'folders' variable
    # eg.
    # folders={0:"black", 1:"blond", 2:"brown", 3:"gray"}
    folders=os.listdir(path_to_val_img)
    
    for i in range(0,len(folders)):
        files=os.listdir(path_to_val_img+folders[i]+"/")
        for file in files:
            write_this=[path_to_val_img+folders[i]+"/"+file,i]
            csv_writer.writerow(write_this)

#### Preparing X_train

In [11]:
def get_data_from_local(X_train):
    X_data =[]
    for file in tqdm(X_train['path']):
        face = cv2.imread(file,cv2.IMREAD_COLOR)
        face =cv2.resize(face, (218,178)) # dimensions (height, width) in witch the model was trained
        face = np.asarray(face)
        face = face.astype('float32')
        face /= 255
        X_data.append(face)
    return np.array(X_data)

#### Preparing the X_test and y_test

In [16]:
file_path="./dataset.csv"
x_columns_for_remove="hair_color" # change according to the label column header in Dataset.csv 
y_columns_for_remove="path" # no need to change
training_feature = 'hair_color'

dataset = pd.read_csv(file_path)
X_test = dataset.drop(columns=x_columns_for_remove)
y_test = dataset.drop(columns=y_columns_for_remove)

y_test = np_utils.to_categorical(y_test)
X_test = get_data_from_local(X_test)
y_test = np.asarray(y_test).astype(np.float32)
print('Shape of Dataset::',training_feature,X_test.shape,y_test.shape)  

100%|███████████████████████████████████████████████████████████████████████████████| 400/400 [00:00<00:00, 721.34it/s]


Shape of Dataset:: hair_color (400, 178, 218, 3) (400, 4)


#### Importing the saved model

In [20]:
path_of_model = 'final_cnn_model_checkpoint.tf' # model checkpoint filename
model = load_model(path_of_model)

#### Scoring

In [24]:
scores = model.evaluate(X_test, y_test, verbose=1)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

13/13 [==============================] - 13s 963ms/step - loss: 2.6427 - accuracy: 0.3975
Baseline Error: 60.25%
